In [1]:
import requests

def get_script(url):
    api_endpoint = "http://127.0.0.1:8010/extract_info"
    response = requests.get(api_endpoint, params={"url": url})
    return response.json()

In [2]:
import time
start_time = time.time()
result = get_script("https://youtu.be/eMlx5fFNoYc?si=V2bsd5GtXo1FhT68")
print(f"실행시간: {time.time() - start_time} seconds")

실행시간: 56.281145334243774 seconds


In [3]:
result.get("script")[:2]

[{'text': 'In the last chapter, you and I started to step through the internal workings of a transformer.',
  'start': 0.12,
  'end': 4.66},
 {'text': 'This is one of the key pieces of technology inside large language models, and a lot of',
  'start': 4.66,
  'end': 8.52}]

In [4]:
import json
import os
if os.path.exists('script.json'):
    os.remove('script.json')

with open('script.json', 'w', encoding='utf-8') as json_file:
    json.dump(result['script'], json_file, ensure_ascii=False, indent=4)

In [5]:
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings(action='ignore')

load_dotenv()  # .env 파일에서 환경 변수를 로드합니다
api_key = os.getenv("OPENAI_API_KEY")

In [6]:
import json
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_teddynote.callbacks import StreamingCallback
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document

summary_prompt = hub.pull("teddynote/summary-stuff-documents-korean")
summary_prompt.template = 'Please summarize the sentence according to the following REQUEST.\nREQUEST:\n1. Summarize the main points in bullet points in KOREAN.\n2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.\n3. Use various emojis to make the summary more interesting.\n4. Translate the summary into KOREAN if it is written in ENGLISH.\n5. DO NOT translate any technical terms.\n6. DO NOT include any unnecessary information.\n7. Please refer to each summary and indicate the key topic.\n\nCONTEXT:\n{context}\n\nSUMMARY:"\n'
summary_prompt.pretty_print()


llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    streaming=True,
    temperature=0.7,
    callbacks=[StreamingCallback()],
)

# Step 1: 파일 로드 및 데이터 변환
def load_script(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        script_data = json.load(file)
    return script_data

Please summarize the sentence according to the following REQUEST.
REQUEST:
1. Summarize the main points in bullet points in KOREAN.
2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.
3. Use various emojis to make the summary more interesting.
4. Translate the summary into KOREAN if it is written in ENGLISH.
5. DO NOT translate any technical terms.
6. DO NOT include any unnecessary information.
7. Please refer to each summary and indicate the main topic.

CONTEXT:
{context}

SUMMARY:"



In [49]:
from langchain_community.document_loaders import JSONLoader,TextLoader

# JSONLoader로 임시 파일 로드
# jq_schema = '.[] | {content: .text, start_time: .start, end_time: .end}'
jq_schema = ".[].text"
docs = JSONLoader(file_path="script.json", jq_schema=jq_schema,text_content=False).load()

# # 각 Document의 page_content를 디코딩하여 한글로 변환
# for doc in docs:
#     # page_content를 JSON 객체로 변환 후 문자열 디코딩
#     decoded_content = json.loads(doc.page_content) 
#     doc.page_content = decoded_content # JSON 문자열 -> Python dict 변환

In [50]:
docs[0]

Document(metadata={'source': '/home/jinu/my_ws/youtube_script_chatbot/script.json', 'seq_num': 1}, page_content='In the last chapter, you and I started to step through the internal workings of a transformer.')

In [51]:
with open("script.txt","w") as file:
    for text in [doc.page_content for doc in docs]:
        file.write(text+'\n')

In [52]:
docs = TextLoader("script.txt").load()

In [53]:
len(docs[0].page_content)

27899

In [12]:
stuff_chain = create_stuff_documents_chain(llm, summary_prompt)
answer = stuff_chain.invoke({"context": docs})

### Main Topic: Transformer와 Attention Mechanism의 작동 원리

- 🌟 **Transformer의 중요성**: Transformer는 대형 언어 모델과 현대 AI 도구의 핵심 기술로, 2017년 논문 "Attention is All You Need"에서 처음 소개됨.
  
- 📄 **모델의 목표**: 모델은 주어진 텍스트를 입력받아 다음에 올 단어를 예측하는 것이 목표이며, 텍스트는 토큰으로 나뉘어 처리됨.

- 🔢 **임베딩과 의미**: 각 토큰은 고차원 벡터(임베딩)와 연관되어 있으며, 이 벡터의 방향은 의미를 나타낼 수 있음.

- 🔍 **Attention Mechanism의 필요성**: 단어의 의미는 문맥에 따라 달라지므로, Attention 메커니즘이 각 단어의 의미를 업데이트하는 과정이 필요함.

- 🧩 **단일 Attention Head**: 단일 Attention Head는 키, 쿼리, 값의 세 가지 행렬을 사용하여 단어의 의미를 조정하며, 각 단어는 자신의 쿼리 벡터를 통해 문맥에서 관련 단어를 찾음.

- 📊 **Attention 패턴**: 쿼리와 키의 점곱을 통해 관련성 점수를 계산하고, 이를 소프트맥스를 통해 정규화하여 주의 패턴을 형성함.

- 🔄 **Multi-Headed Attention**: 다수의 Attention Head를 병렬로 실행하여 다양한 문맥 정보가 반영된 의미 업데이트를 수행함.

- 📈 **모델 파라미터**: GPT-3의 경우, 96개의 Attention Head가 존재하여 모델의 총 파라미터 수는 약 58억에 달함.

- ⚡ **병렬 처리의 장점**: Attention 메커니즘은 매우 병렬화가 가능하여 대량의 계산을 신속하게 수행할 수 있는 장점이 있음.

- 📚 **추가 자료**: 더 많은 정보를 원한다면 Andre Karpathy, Chris Ola 등의 자료를 참고하는 것이 유용함.

In [96]:
template = """REQUEST:
1. Separate the entire content based on the main topics and group sentences into logical paragraphs.
2. Each paragraph should be structured as a key-value pair in a dictionary. 
3. The keys should be "paragraph1", "paragraph2", and so on. The values should be the corresponding paragraph text.
4. Each sentence in the paragraph should be separated by a newline character.
5. Do not distort the original content under any circumstances.

CONTEXT:
{context}

RESULT:"""

In [110]:
paragraph_separate_template = PromptTemplate(input_variables=['context'],input_types={},partial_variables={},metadata={'lc_hub_owner':"jinu","lc_hub_repo":None,"lc_hub_commit_hash":None},template=template)

In [ ]:
stuff_chain = create_stuff_documents_chain(llm, paragraph_separate_template)
answer = stuff_chain.invoke({"context": docs})

In [114]:
import ast
text_data = answer[answer.find("{"):answer.find("}")+1]
parsed_data = ast.literal_eval(text_data)
parsed_data

{'paragraph1': 'In the last chapter, you and I started to step through the internal workings of a transformer.\nThis is one of the key pieces of technology inside large language models, and a lot of other tools in the modern wave of AI.\nIt first hit the scene in a now-famous 2017 paper called Attention is All You Need, and in this chapter, you and I will dig into what this attention mechanism is, visualizing how it processes data.',
 'paragraph2': "As a quick recap, here's the important context I want you to have in mind.\nThe goal of the model that you and I are studying is to take in a piece of text and predict what word comes next.\nThe input text is broken up into little pieces that we call tokens, and these are very often words or pieces of words, but just to make the examples in this video easier for you and me to think about, let's simplify by pretending that tokens are always just words.",
 'paragraph3': 'The first step in a transformer is to associate each token with a high-d

In [13]:
for line in answer.strip().split("\n"):
    # 줄이 비어 있지 않다면 Document로 생성하고, 요약 내용임을 metadata로 추가
    if line.strip():
        docs[0].page_content += line+"\n"

In [14]:
splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
split_docs = splitter.split_documents(docs)

In [15]:
embeddings = OpenAIEmbeddings()

In [16]:
vec_store = FAISS.from_documents(split_docs, embeddings)

In [17]:
bm25_retriever = BM25Retriever.from_documents(split_docs)
bm25_retriever.k=10

In [18]:
vec_retriever = vec_store.as_retriever( search_kwargs={"k": 10} )

In [19]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[
        bm25_retriever,
        vec_retriever,
    ],
    weights=[0.7, 0.3],
)

In [46]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
chat_prompt = PromptTemplate.from_template(
    """당신은 유튜브 스크립트 기반의 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 
당신의 임무는 주어진 영상의 텍스트 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 
만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)


In [47]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": ensemble_retriever, "question": RunnablePassthrough()}
    | chat_prompt
    | llm
    | StrOutputParser()
)

In [115]:
query = "병렬처리가 장점은 뭔가요?"
ret_docs = ensemble_retriever.invoke(query)
for doc in ret_docs:
    print(doc.page_content)
print("-"*100)
response = chain.invoke(query)

- 📚 **추가 자료**: 더 많은 정보를 원한다면 Andre Karpathy, Chris Ola 등의 자료를 참고하는 것이 유용함.
I like to visualize a grid full of a bunch of dots, where the bigger dots correspond
example,
that would look a little more like this, where if the keys produced by fluffy and blue really do
align closely with the query produced by creature, then the dot products in these two spots would
would be
some large positive numbers. In the lingo, machine learning people would say that this means the
embeddings of fluffy and blue attend to the embedding of creature. By contrast to the dot
product between the key for some other word like the and the query for creature would be some small
or negative value that reflects that these are unrelated to each other. So we have this grid of
- ⚡ **병렬 처리의 장점**: Attention 메커니즘은 매우 병렬화가 가능하여 대량의 계산을 신속하게 수행할 수 있는 장점이 있음.
- 🔄 **Multi-Headed Attention**: 다수의 Attention Head를 병렬로 실행하여 다양한 문맥 정보가 반영된 의미 업데이트를 수행함.
자신의 쿼리 벡터를 통해 문맥에서 관련 단어를 찾음.
- 🔍 **Attention Mechanism의 필요성**: 단어의 의미는 문맥에 